Para este ejercicio lo primero que se debe realizar es la instalación de las librerías PyPDF2 y openai. Esto para importar las hojas de vida. Para este ejercicio he tenido que simular con hojas de vida personales ya que no encontré el zip con las del enunciado. Sin embargo funcionaría de igual forma, solo incluyendolas dentro de la carpeta destinada en el colab  

In [1]:
#Importe de librerías necesarias

!pip install spacy transformers
!pip install PyPDF2
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00


Posteriormente se realiza el importe de las librerías mencionadas anteriormente, además de os, para trasladarse dentro del directorio activo y json para poder generar el json final.

In [65]:
# Importe de librerías
import os
import json
import PyPDF2
import openai

Para este ejercicio se ha probrado un prompt zero shoot con openai, para esto se utiliza el api key del proceso. También se utiliza la carpeta del directorio en donde irán colocadas las hojas de vida para su diagnóstico, y se define el array de all_results para poder acumular la información de los json que se van recopilando.

In [72]:
# Configurar la API de OpenAI
openai.api_key = 'no se comparte por seguridad'

# Ruta a la carpeta que contiene los archivos PDF
pdf_folder_path = '/content/CV/'

# Lista para almacenar los resultados JSON
all_results = []

A continuación, se genera una iteración para poder recorrer cada uno de los elementos de la carpeta en donde se encuentran las hojas de vida. En este proceso, se abre el pdf, se extrae el texto y luego se introduce a un modelo de inteligencia artificial generativa de openai. Se intento primero con el modelo de completion (gpt-3.5-turbo-instruct), pero no generó los resultados esperados, por lo que se decidió utilizar la ia de chat completions (gpt-3.5-turbo-1106). Inicialmente se pensó en el de completions, porque era una acción definida la que se ejecutaría, mas no un chat, pero lamentablemente el json no quedaba bien definido. En este caso se asegura que el JSON quede bien construido a base de un prompt procedimental zero shoot, pero con la isntrucción clara de como construir el json. La salida del json estaba generando dificultadas y se intentó generar de diversas maneras. La mas efectiva fue la utilizada actualmente, lo que obligó a extraer el json por medio de expresiones regulares.

In [73]:
# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(pdf_folder_path):
    if filename.endswith('.pdf'):
        pdf_file_path = os.path.join(pdf_folder_path, filename)

        # Leer el contenido del archivo PDF
        with open(pdf_file_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ''
            for page in reader.pages:
                text += page.extract_text()

        # Llamada a la API de OpenAI con el modelo de chat completions
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {"role": "system", "content": "Eres un asistente que extrae datos de currículums."},
                {"role": "user", "content": f"""
                Dado el siguiente texto:

                \"{text}\"

                1. Debes memorizar solo los siguientes datos del texto:
                   - Nombre
                   - Teléfono
                   - Cédula
                   - Años de experiencia

                2. Basado en la información proporcionada, deberás tener en cuenta si el candidato tiene un background en inteligencia artificial (IA) o no.

                3. Tu respuesta debe ser únicamente un JSON que contenga solo esta información:

                ```json
                {{
                "nombre": "<nombre extraído>",
                "telefono": "<teléfono extraído>",
                "cedula": "<cédula extraída>",
                "años de experiencia": "<Solo el número de años de experiencia extraídos>",
                "background IA": "<Solo hay dos opciones de respuesta: 'Sí' o 'No'>"
                }}
                ```
                """}
            ],
            max_tokens=200
        )

        # Procesar la respuesta de la API
        output = response.choices[0].message['content'].strip()
        print(output)

        # Extraer solo el contenido JSON utilizando expresión regular
        json_match = re.search(r'\{.*?\}', output, re.DOTALL)
        if json_match:
            json_content = json_match.group(0)
        else:
            json_content = '{"error": "No se encontró contenido JSON"}'

        print("Contenido JSON extraído:", json_content)


        # Convertir la salida en un diccionario JSON
        try:
            json_output = json.loads(json_content)
        except json.JSONDecodeError:
            print(f"Error al decodificar JSON para el archivo: {filename}")
            json_output = {"error": "Invalid JSON format"}

        # Agregar el resultado a la lista
        json_output["archivo"] = filename
        all_results.append(json_output)

```json
{
  "nombre": "Diana Fernanda Serna Guzmán",
  "telefono": "3135536157",
  "cedula": "1.053.843.025",
  "años de experiencia": "4",
  "background IA": "No"
}
```
Contenido JSON extraído: {
  "nombre": "Diana Fernanda Serna Guzmán",
  "telefono": "3135536157",
  "cedula": "1.053.843.025",
  "años de experiencia": "4",
  "background IA": "No"
}
```json
{
  "nombre": "Andrés Felipe Serna Guzmán",
  "telefono": "3127964371",
  "cedula": "1053830076",
  "años de experiencia": "6",
  "background IA": "Sí"
}
```
Contenido JSON extraído: {
  "nombre": "Andrés Felipe Serna Guzmán",
  "telefono": "3127964371",
  "cedula": "1053830076",
  "años de experiencia": "6",
  "background IA": "Sí"
}
```json
{
  "nombre": "Juan Sebastian Correa Giraldo",
  "telefono": "3173750355",
  "cedula": null,
  "años de experiencia": "6",
  "background IA": "No"
}
```
Contenido JSON extraído: {
  "nombre": "Juan Sebastian Correa Giraldo",
  "telefono": "3173750355",
  "cedula": null,
  "años de experiencia"

Por ultimo, con los json listos para ser guardados, se utiliza json dump y se guardan los archivos de tal manera de que quedan almacenados dentro del proceso actual.

In [74]:
# Guardar todos los resultados en un archivo JSON
with open('/content/CV/resultados_combinados.json', 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

print("Todos los resultados se han guardado en 'resultados_combinados.json'.")

Todos los resultados se han guardado en 'resultados_combinados.json'.
